In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import string
import re
%matplotlib inline
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\st

Raw Data

In [2]:
edu = pd.read_csv('education_tweets.csv')
# edu = edu.dropna()
edu.head()

,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source
0,0,ISA - The International Schools Association,"Geneve, Switzerlad","A non-governmental international organization,...",70,3092078225,2022-11-18 19:29:29+00:00,We want to congratulate Colegio Lakeside for h...,[None],Metricool
1,1,onlineCourses,NaN,Sending out free coupons to Online web develop...,86,2591443279,2022-11-18 19:29:01+00:00,#JAVASCRIPT#COURSES #LEARN #FREE JavaScript DO...,"[['LEARN', 'FREE']]",FreeCourses
2,2,SCarolina Education,South Carolina,Follow this account for geo-targeted Education...,0,181076039,2022-11-18 19:28:41+00:00,See our latest SC #Education job opportunity a...,[['Education']],CareerArc 2.0
3,3,Ciara Quill,Ireland,Teacher & lecturer of 🇫🇷 🇩🇪 Interests include ...,74683,1920750644,2022-11-18 19:28:29+00:00,#teachers #education #schools And yet the Depa...,"[['teachers', 'education', 'schools']]",Twitter for iPhone
4,4,Jeremy Yanofsky,"California, USA",Champion for #gamebasedlearning @ https://t.co...,3930,321521246,2022-11-18 19:28:26+00:00,While so many of my colleagues in #education a...,"[['education', 'RIPTwitter', 'PLN']]",Twitter Web App


In [3]:
len(edu)

10000

Roberta Analysis- Raw

In [4]:
rob_edu = list()
# new =pd.DataFrame()
for i,  row in tqdm(edu.iterrows(), total = len(edu)):
    try:
#         print({i})
        text = row['text']
        encoded_text= tokenizer(text, return_tensors = 'pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg' : scores[0],
            'roberta_neu' : scores[1],
            'roberta_pos' : scores[2]
        }
        rob_edu.append(scores_dict) 
    except RuntimeError:
        print(f'Too Long Text')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
rob_df = pd.DataFrame(rob_edu)
rob_df

,roberta_neg,roberta_neu,roberta_pos
0,0.001225,0.043884,0.954890
1,0.009886,0.408936,0.581178
2,0.013409,0.909351,0.077240
3,0.369919,0.571984,0.058097
4,0.044339,0.285380,0.670281
...,...,...,...
9995,0.003705,0.103956,0.892339
9996,0.002441,0.052594,0.944965
9997,0.001294,0.067178,0.931528
9998,0.030144,0.881462,0.088394


In [6]:
rob_edu2 = rob_df.join(edu)

In [7]:
rob_edu2

,roberta_neg,roberta_neu,roberta_pos,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source
0,0.001225,0.043884,0.954890,0,ISA - The International Schools Association,"Geneve, Switzerlad","A non-governmental international organization,...",70,3092078225,2022-11-18 19:29:29+00:00,We want to congratulate Colegio Lakeside for h...,[None],Metricool
1,0.009886,0.408936,0.581178,1,onlineCourses,NaN,Sending out free coupons to Online web develop...,86,2591443279,2022-11-18 19:29:01+00:00,#JAVASCRIPT#COURSES #LEARN #FREE JavaScript DO...,"[['LEARN', 'FREE']]",FreeCourses
2,0.013409,0.909351,0.077240,2,SCarolina Education,South Carolina,Follow this account for geo-targeted Education...,0,181076039,2022-11-18 19:28:41+00:00,See our latest SC #Education job opportunity a...,[['Education']],CareerArc 2.0
3,0.369919,0.571984,0.058097,3,Ciara Quill,Ireland,Teacher & lecturer of 🇫🇷 🇩🇪 Interests include ...,74683,1920750644,2022-11-18 19:28:29+00:00,#teachers #education #schools And yet the Depa...,"[['teachers', 'education', 'schools']]",Twitter for iPhone
4,0.044339,0.285380,0.670281,4,Jeremy Yanofsky,"California, USA",Champion for #gamebasedlearning @ https://t.co...,3930,321521246,2022-11-18 19:28:26+00:00,While so many of my colleagues in #education a...,"[['education', 'RIPTwitter', 'PLN']]",Twitter Web App
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.003705,0.103956,0.892339,9995,Lux Vincit Official,NaN,Conspiracy-History content.\nDaily uploads.,0,1592908819910246408,2022-11-16 15:58:42+00:00,Lux Vincit now has Twitter🌟 Be sure to follow ...,"[['twitter', 'History', 'education']]",Twitter for Android
9996,0.002441,0.052594,0.944965,9996,Joe Sweeney,Wolverhampton,Local Democracy Reporter for Wolverhampton in ...,369,1029297496768634880,2022-11-16 15:58:20+00:00,'Delightful and inspiring' Wolverhampton nurse...,[['LDReporter']],Twitter Web App
9997,0.001294,0.067178,0.931528,9997,Dedan Brozino 🌹🇺🇸🌹,"Pasadena, CA",God + Family | Chief Development Officer at @R...,17443,611740066,2022-11-16 15:58:16+00:00,Important and impactful discussion yesterday i...,[None],Twitter for iPhone
9998,0.030144,0.881462,0.088394,9998,HA Trade Talk,NaN,# SEBI registered Authorized sub broker\n# Tra...,932,1503382250150576129,2022-11-16 15:58:01+00:00,BEL signed separate MoUs with Hindustan Shipya...,[None],Twitter Web App


VADERS Analyis

In [3]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

In [4]:
# res = {}
v_edu = list()
# new =pd.DataFrame()
for i,  row in tqdm(edu.iterrows(), total = len(edu)):
    text = row['text']
#     myid = row['reviewerID']
    res = sia.polarity_scores(text)
#     print(res)
    v_edu.append(res)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
len(v_edu)

10000

In [6]:
vaders_edu = pd.DataFrame(v_edu)
vaders_edu

,neg,neu,pos,compound
0,0.00,0.847,0.153,0.7639
1,0.00,0.804,0.196,0.7783
2,0.00,0.812,0.188,0.7184
3,0.00,0.889,0.111,0.3612
4,0.13,0.806,0.064,-0.4939
...,...,...,...,...
9995,0.00,0.758,0.242,0.7712
9996,0.00,0.485,0.515,0.8910
9997,0.00,0.766,0.234,0.8316
9998,0.00,0.952,0.048,0.1027


In [9]:
z = pd.read_csv('education_tweets_analysis.csv')
z

,Unnamed: 0.1,neg,neu,pos,compound,roberta_neg,roberta_neu,roberta_pos,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source
0,0,0.00,0.847,0.153,0.7639,0.001225,0.043884,0.954890,0,ISA - The International Schools Association,"Geneve, Switzerlad","A non-governmental international organization,...",70,3092078225,2022-11-18 19:29:29+00:00,We want to congratulate Colegio Lakeside for h...,[None],Metricool
1,1,0.00,0.804,0.196,0.7783,0.009886,0.408936,0.581178,1,onlineCourses,NaN,Sending out free coupons to Online web develop...,86,2591443279,2022-11-18 19:29:01+00:00,#JAVASCRIPT#COURSES #LEARN #FREE JavaScript DO...,"[['LEARN', 'FREE']]",FreeCourses
2,2,0.00,0.812,0.188,0.7184,0.013409,0.909351,0.077240,2,SCarolina Education,South Carolina,Follow this account for geo-targeted Education...,0,181076039,2022-11-18 19:28:41+00:00,See our latest SC #Education job opportunity a...,[['Education']],CareerArc 2.0
3,3,0.00,0.889,0.111,0.3612,0.369919,0.571984,0.058097,3,Ciara Quill,Ireland,Teacher & lecturer of 🇫🇷 🇩🇪 Interests include ...,74683,1920750644,2022-11-18 19:28:29+00:00,#teachers #education #schools And yet the Depa...,"[['teachers', 'education', 'schools']]",Twitter for iPhone
4,4,0.13,0.806,0.064,-0.4939,0.044339,0.285380,0.670281,4,Jeremy Yanofsky,"California, USA",Champion for #gamebasedlearning @ https://t.co...,3930,321521246,2022-11-18 19:28:26+00:00,While so many of my colleagues in #education a...,"[['education', 'RIPTwitter', 'PLN']]",Twitter Web App
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.00,0.758,0.242,0.7712,0.003705,0.103956,0.892339,9995,Lux Vincit Official,NaN,Conspiracy-History content.\nDaily uploads.,0,1592908819910246408,2022-11-16 15:58:42+00:00,Lux Vincit now has Twitter🌟 Be sure to follow ...,"[['twitter', 'History', 'education']]",Twitter for Android
9996,9996,0.00,0.485,0.515,0.8910,0.002441,0.052594,0.944965,9996,Joe Sweeney,Wolverhampton,Local Democracy Reporter for Wolverhampton in ...,369,1029297496768634880,2022-11-16 15:58:20+00:00,'Delightful and inspiring' Wolverhampton nurse...,[['LDReporter']],Twitter Web App
9997,9997,0.00,0.766,0.234,0.8316,0.001294,0.067178,0.931528,9997,Dedan Brozino 🌹🇺🇸🌹,"Pasadena, CA",God + Family | Chief Development Officer at @R...,17443,611740066,2022-11-16 15:58:16+00:00,Important and impactful discussion yesterday i...,[None],Twitter for iPhone
9998,9998,0.00,0.952,0.048,0.1027,0.030144,0.881462,0.088394,9998,HA Trade Talk,NaN,# SEBI registered Authorized sub broker\n# Tra...,932,1503382250150576129,2022-11-16 15:58:01+00:00,BEL signed separate MoUs with Hindustan Shipya...,[None],Twitter Web App


In [2]:
# z['reviewtext'] = z['text'].apply(lambda text:sia.polarity_scores(str(text)))

In [3]:
# z['reviewText']

Combine both datasets

In [5]:
# final_df = vaders_edu.join(rob_edu2)

In [ ]:
final_df

In [ ]:
# tweets_df.to_csv('education_tweets.csv')

Clean Data

In [85]:
edu = pd.read_csv('education_tweets_cleaned.csv')

In [86]:
# # Remove Punctuations
# def remove_punct(text):
#     text  = "".join([char for char in text if char not in string.punctuation])
#     text = re.sub('[0-9]+', '', text)
#     return text

# edu['text2'] = edu['text'].apply(lambda x: remove_punct(x))

# edu

In [87]:
# def tokenization(text):
#     text = re.split('\W+', text)
#     return text

# edu['text3'] = edu['text2'].apply(lambda x: tokenization(x.lower()))
# edu.head()

In [89]:
# stopword = nltk.corpus.stopwords.words('english')
# def remove_stopwords(text):
#     text = [word for word in text if word not in stopword]
#     return text
    
# edu['text4'] = edu['text3'].apply(lambda x: remove_stopwords(x))


In [90]:
# edu.to_csv('education_tweets_cleaned.csv')

In [91]:
rob_edu = list()
# new =pd.DataFrame()
for i,  row in tqdm(edu.iterrows(), total = len(edu)):
    try:
#         print({i})
        text = row['text4']
        encoded_text= tokenizer(text, return_tensors = 'pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg' : scores[0],
            'roberta_neu' : scores[1],
            'roberta_pos' : scores[2]
        }
        rob_edu.append(scores_dict) 
    except RuntimeError:
        print(f'Too Long Text')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [95]:
rob_df3 = pd.DataFrame(rob_edu)

In [96]:
rob_edu2 = rob_df3.join(edu)

In [98]:
rob_edu2['text5'] = rob_edu2['text4'].apply(lambda text4:sia.polarity_scores(str(text4)))

objs = [rob_edu2, pd.DataFrame(rob_edu2['text5'].tolist()).iloc[:, :4]]
rob_edu3 = pd.concat(objs, axis=1).drop('text5', axis=1)

In [99]:
rob_edu3

,roberta_neg,roberta_neu,roberta_pos,Unnamed: 0.1,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,...,text,hashtags,source,text2,text3,text4,neg,neu,pos,compound
0,0.001760,0.107814,0.890425,0,0,ISA - The International Schools Association,"Geneve, Switzerlad","A non-governmental international organization,...",70,3.092078e+09,...,We want to congratulate Colegio Lakeside for h...,[None],Metricool,We want to congratulate Colegio Lakeside for h...,we want to congratulate colegio lakeside for h...,want congratulate colegio lakeside hosting you...,0.00,0.774,0.226,0.7184
1,0.017012,0.634304,0.348684,1,1,onlineCourses,NaN,Sending out free coupons to Online web develop...,86,2.591443e+09,...,#JAVASCRIPT#COURSES #LEARN #FREE JavaScript DO...,"[['LEARN', 'FREE']]",FreeCourses,JAVASCRIPTCOURSES LEARN FREE JavaScript DOM Dy...,javascriptcourses learn free javascript dom dy...,javascriptcourses learn free javascript dom dy...,0.00,0.644,0.356,0.9246
2,0.016912,0.905337,0.077751,2,2,SCarolina Education,South Carolina,Follow this account for geo-targeted Education...,0,1.810760e+08,...,See our latest SC #Education job opportunity a...,[['Education']],CareerArc 2.0,See our latest SC Education job opportunity an...,see our latest sc education job opportunity an...,see latest sc education job opportunity click ...,0.00,0.739,0.261,0.7184
3,0.108545,0.850891,0.040564,3,3,Ciara Quill,Ireland,Teacher & lecturer of 🇫🇷 🇩🇪 Interests include ...,74683,1.920751e+09,...,#teachers #education #schools And yet the Depa...,"[['teachers', 'education', 'schools']]",Twitter for iPhone,teachers education schools And yet the Departm...,teachers education schools and yet the departm...,teachers education schools yet department educ...,0.00,0.828,0.172,0.3612
4,0.011439,0.176334,0.812228,4,4,Jeremy Yanofsky,"California, USA",Champion for #gamebasedlearning @ https://t.co...,3930,3.215212e+08,...,While so many of my colleagues in #education a...,"[['education', 'RIPTwitter', 'PLN']]",Twitter Web App,While so many of my colleagues in education ar...,while so many of my colleagues in education ar...,many colleagues education scrambling riptwitte...,0.14,0.753,0.106,-0.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.007110,0.136402,0.856488,9995,9995,Lux Vincit Official,NaN,Conspiracy-History content.\nDaily uploads.,0,1.592910e+18,...,Lux Vincit now has Twitter🌟 Be sure to follow ...,"[['twitter', 'History', 'education']]",Twitter for Android,Lux Vincit now has Twitter🌟 Be sure to follow ...,lux vincit now has twitter be sure to follow m...,lux vincit twitter sure follow plenty great co...,0.00,0.686,0.314,0.7506
9996,0.001502,0.028187,0.970311,9996,9996,Joe Sweeney,Wolverhampton,Local Democracy Reporter for Wolverhampton in ...,369,1.029300e+18,...,'Delightful and inspiring' Wolverhampton nurse...,[['LDReporter']],Twitter Web App,Delightful and inspiring Wolverhampton nursery...,delightful and inspiring wolverhampton nursery...,delightful inspiring wolverhampton nursery rat...,0.00,0.459,0.541,0.8910
9997,0.001887,0.072235,0.925878,9997,9997,Dedan Brozino 🌹🇺🇸🌹,"Pasadena, CA",God + Family | Chief Development Officer at @R...,17443,6.117401e+08,...,Important and impactful discussion yesterday i...,[None],Twitter for iPhone,Important and impactful discussion yesterday i...,important and impactful discussion yesterday i...,important impactful discussion yesterday roseb...,0.00,0.660,0.340,0.8316
9998,0.027458,0.868429,0.104113,9998,9998,HA Trade Talk,NaN,# SEBI registered Authorized sub broker\n# Tra...,932,1.503380e+18,...,BEL signed separate MoUs with Hindustan Shipya...,[None],Twitter Web App,BEL signed separate MoUs with Hindustan Shipya...,bel signed separate mous with hindustan shipya...,bel signed separate mous hindustan shipyard us...,0.00,0.849,0.151,0.4767


In [101]:
rob_edu3.to_csv('education_tweets_analysis_cleaned.csv')